In [14]:
import pandas as pd
import joblib
import time

from sklearn.model_selection import train_test_split
from skopt import BayesSearchCV  # Bayesian optimization: utilizado para optimizar hiperparámetros

import lightgbm as lgbm
from lightgbm import early_stopping  # Early stopping: utilizado para evitar sobreajuste

from Funcoes_Comuns import avaliar_modelo, registrar_modelo

In [15]:
# Obter dados
df_enem = pd.read_pickle('Bases\Finais\enem_censo_2023_full.pkl')

In [16]:
variaveis_alvo = ['NUM_NOTA_MT', 'NUM_NOTA_LC', 'NUM_NOTA_CN', 'NUM_NOTA_CH', 'NUM_NOTA_REDACAO']

# separar em treino e teste
X = df_enem.drop(columns=variaveis_alvo)
y = df_enem[variaveis_alvo]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
# Ajuste de tipo para MLflow
# Converter colunas inteiras para float
X_train = X_train.astype({col: 'float' for col in X_train.select_dtypes('int').columns})
X_test = X_test.astype({col: 'float' for col in X_test.select_dtypes('int').columns})

Aplicar Bayes Search

In [18]:
# Definição do espaço de busca para otimização bayesiana
param_grid = {
    'num_leaves': (5, 60),                         # Número de folhas na árvore de decisão
    'max_depth': (40, 100),                        # Profundidade máxima da árvore
    'learning_rate': (0.005, 0.1, 'log-uniform'),  # Taxa de aprendizado
    'n_estimators': (5000, 6000),                  # Número de árvores
    'subsample': (0.3, 1.0),                       # Proporção de amostras usadas em cada árvore
    'colsample_bytree': (0.2, 1.0),                # Fração de colunas a serem usadas por árvore
    'reg_alpha': (1e-3, 1.0, 'log-uniform'),       # Regularização L1
    'reg_lambda': (1e-5, 1.0, 'log-uniform'),      # Regularização L2
}

In [19]:
def busca_bayesiana_lgbm_por_nota(X_train: pd.DataFrame, 
                                    y_train: pd.DataFrame, 
                                    X_test: pd.DataFrame,
                                    y_test: pd.DataFrame,
                                    nota: str,
                                    modelo_lgbm_bayes: lgbm.LGBMRegressor,
                                    param_grid: dict):
    """
    Realiza a busca Bayesiana para otimização de hiperparâmetros do modelo LGBMRegressor.
    
    Parâmetros:
    - X_train: DataFrame com as variáveis independentes de treino.
    - y_train: DataFrame com as variáveis dependentes de treino.
    - X_test: DataFrame com as variáveis independentes de teste.
    - y_test: DataFrame com as variáveis dependentes de teste.
    - nota: Nome da variável alvo a ser otimizada (ex: 'NUM_NOTA_MT').
    - modelo_lgbm_bayes: Instância do modelo LGBMRegressor a ser otimizado.
    - param_grid: Dicionário com os hiperparâmetros a serem otimizados.
    Retorna:
    - None: A função registra o modelo otimizado no MLflow e salva o modelo treinado.
    """
    
    # Configurar a busca Bayesiana usando BayesSearchCV
    # Criando o otimizador Bayesiano
    bayes_search = BayesSearchCV(
        estimator=modelo_lgbm_bayes,    # Modelo a ser otimizado
        search_spaces=param_grid,       # Espaço de busca definido acima
        scoring='r2',                   # Critério de seleção
        n_iter=30,                      # Número de avaliações do modelo
        cv=5,                           # Validação cruzada
        random_state=42,                # Semente para reprodutibilidade
        n_jobs=-1,                      # Paralelização total dos cálculos
        verbose=1                       # 0 = sem mensagens, 1 = mensagens de progresso, 2 = mensagens detalhadas
    )

    # Criar Eval Set para validação cruzada (15% do conjunto de treino)
    X_train_bayes, X_eval, y_train_bayes, y_eval = train_test_split(
        X_train,
        y_train[nota],  # Usando apenas a variável alvo NUM_NOTA_MT para otimização
        test_size=0.15,
        random_state=42
    )

    categorical_features = X_train.select_dtypes(include=['category']).columns.tolist()

    # Parametros validação para o modelo
    fit_params = {
        'eval_set': [(X_eval, y_eval)],                    # Conjunto de validação
        'eval_metric': ['r2', 'rmse', 'mae'],              # Métricas a serem avaliadas
        'categorical_feature': categorical_features,       # Colunas categóricas
        'callbacks': [early_stopping(stopping_rounds=50)]  # Early stopping para evitar sobreajuste
    }

    # Executar a busca Bayesiana
    start_time = time.time()
    bayes_search.fit(X_train_bayes, y_train_bayes, **fit_params)
    # Parar o cronômetro
    end_time = time.time()
    elapsed_time = end_time - start_time

    # Resultados da busca Bayesiana
    print("Melhores parâmetros encontrados:")
    print(bayes_search.best_params_)
    print("R2: ", bayes_search.best_score_)
    print("Tempo de execução da busca Bayesiana: {:.2f} segundos".format(elapsed_time))

    # Treinar o modelo com os melhores parâmetros encontrados
    modelo_lgbm_bayes.set_params(**bayes_search.best_params_)
    start_time = time.time()
    # Treinamento do modelo com os melhores parâmetros encontrados
    modelo_lgbm_bayes.fit(X_train_bayes, 
                            y_train_bayes, 
                            eval_set=[(X_eval, y_eval)], 
                            eval_metric=['r2', 'rmse', 'mae'],
                            categorical_feature=categorical_features,
                            callbacks=[early_stopping(stopping_rounds=50)]
                        )

    tempo_treino = time.time() - start_time

    # Previsões
    y_pred_bayes = modelo_lgbm_bayes.predict(X_test)

    # registrar o modelo no MLflow
    nome_experimento = 'Notas ENEM + Censo 2023'
    registrar_modelo(experimento=nome_experimento,
                        modelo=modelo_lgbm_bayes,
                        parametros={**modelo_lgbm_bayes.get_params(), "amostra": X_train.shape[0], "tempo": tempo_treino},
                        X_train=X_train,
                        y_train=y_train,
                        y_test=y_test,
                        y_pred=y_pred_bayes,
                        variavel_alvo=nota,
                        nome_modelo=f'modelo_lgbm_bayes_censo_enem_{nota}',
                        descricao_modelo='Modelo LGBMRegressor otimizado com BayesSearchCV Censo e ENEM 2023 para a variável alvo ' + nota)
    
    # Avaliação grupo treino
    avaliar_modelo(y_train[nota], modelo_lgbm_bayes.predict(X_train), "treino")

    # Avaliação grupo teste
    avaliar_modelo(y_test[nota], y_pred_bayes, "teste")

    # Salvar o modelo treinado
    joblib.dump(modelo_lgbm_bayes, f'Modelos/modelo_lgbm_bayes_censo_enem_{nota}.pkl')

In [20]:
notas_alvo = ['NUM_NOTA_MT', 'NUM_NOTA_CH', 'NUM_NOTA_LC', 'NUM_NOTA_CN', 'NUM_NOTA_REDACAO']
notas_alvo = ['NUM_NOTA_CH', 'NUM_NOTA_REDACAO']

for nota in notas_alvo:

    modelo_lgbm_bayes = lgbm.LGBMRegressor(random_state=42,
                                            max_bin=4095, 
                                            force_row_wise=True)
    
    print(f"Iniciando busca Bayesiana para a nota: {nota}")

    busca_bayesiana_lgbm_por_nota(X_train,
                                    y_train,
                                    X_test,
                                    y_test,
                                    nota, 
                                    modelo_lgbm_bayes, 
                                    param_grid)
    
    print(f"Busca Bayesiana concluída para a nota: {nota}\n")

Iniciando busca Bayesiana para a nota: NUM_NOTA_CH
Fitting 5 folds for each of 1 candidates, totalling 5 fits


MemoryError: Unable to allocate 315. MiB for an array with shape (106, 389771) and data type float64